# Introduction to medical statistics
Dr Alan McWilliam

Version 1 - 14th November 2024


---






In this notebook we will work through examples of the most common medical statistics which we introduce in the lecture. We will use R for this practical session. R is an open source scripting language specially designed for statistical analysis. It comes with libraries for data cleaning, survival analysis and classification models, amongst other possibilities.

As this notebook uses R, you will first need to change the runtime type to 'R', in the 'Runtime' drop down menu above, select 'change runtime type'. In the pop-up box, in the drop down list, choose R.



---

### 1. Set-up libraries
First, we need one additional library which isn't available automatically within google colab. This will be used to plot the survival curves.

(This may take a couple minutes...)

In [ ]:
install.packages('survminer')



Now we will call the libraries needed for the script.

In [ ]:
library('survival')
library('ggplot2')
library('survminer')
library('dplyr')



---
### 2. Explore the data
The library 'survival' contains example datasets which we can use to start setting up a survival analysis. Let's load the dataset 'lung', this contains demographic and clinical data from individuals with advanced lung cancer.

First, let's load in the dataset into a varianble called 'lung_data'

In [ ]:
lung_data <- lung


Let's have a look at what is inside the data we have read and start to understand what the data looks like. There will be 10 variables:

1.   nst: Institution code
2.   time: Survival time in days
3.   status: censoring status 1=censored, 2=dead
4.   age: Age in years
5.   sex: Male=1 Female=2
6.   ph.ecog: ECOG performance score as rated by the physician. 0=asymptomatic, 1= symptomatic but completely ambulatory, 2= in bed <50% of the day, 3= in bed > 50% of the day but not bedbound, 4 = bedbound
7.   ph.karno: Karnofsky performance score (bad=0-good=100) rated by physician
8.   pat.karno: Karnofsky performance score (0 = bad, 100 = good) as rated by patient
9.   meal.cal: Calories consumed at meals
10.  wt.loss: Weight loss in last six months


The function **'head'** will print out the first few rows of the dataframe, **'nrow'** will tell us how many rows (i.e., patients) we have available.

In [ ]:
head(lung_data)
nrow(lung_data)

The first thing to do with any dataframe is to start to understand what the dataframe contains and what the data looks like. R has many built in functions to do this with, but we also need to ensure that the data is being handled correctly. For example, does R know if the variables are continuous variables or catagorical?

Let's start by generating a **'summary'** of the dataframe and look to see if the output makes sense.

In [ ]:
summary(lung_data)



Look at the summary statistics. For each we ahve a mean and medium, quartiles, max and min. This is fine for continuous variables but not appropriate for a catagorical variable. For example, sex is 1 for males and 2 for females, here we need a count for each catagory not a mean or medium. Let's update the variable list by changing the type and re-run the summary stats.

In [ ]:
lung_data$status <- factor(lung_data$status)
lung_data$sex <- factor(lung_data$sex)
lung_data$ph.ecog <- factor(lung_data$ph.ecog)
lung_data$ph.karno <- factor(lung_data$ph.karno)
lung_data$pat.karno <- factor(lung_data$pat.karno)

summary(lung_data)

Does this now make more sense?


### 2a. Visualising data
It can also be helpful to start making some plots to look at what the data looks like. The form of plot will depend on the type and shape of the data. The main options being, histograms, box + whisker, violin plots.

You can also use plots to visualise differences between categories. For example in this data, do males or females have greater weight loss?

Let's make some plots and see what we can do...

In [ ]:
# Plot a histogram of participants age in the study
ggplot(data=lung_data, aes(age)) +
  geom_histogram(breaks=seq(30,90, by = 3),
                 col = "skyblue", fill = "lightblue", alpha = 0.5) +
  labs(title = 'Histogram of participates age', x = "Age" ) +
  theme_classic() +
  theme(text = element_text(size = 20))

# Plot histogram of the participants weight loss, split on sex
ggplot(data=lung_data, aes(x = wt.loss, color = sex)) +
  geom_histogram(breaks=seq(-30,70, by = 5), fill="white", position="dodge", alpha = 0.5) +
  labs(title = 'Histogram of weight loss by sex', x = "Age" ) +
  theme_classic() +
  theme(text = element_text(size = 20))


In [ ]:

# Violin plots show us the shape of the data and have many variations, here we will plot the distribution of age grouped on sex
ggplot(lung_data, aes(x=sex, y=age)) +
  geom_violin() +
  stat_summary(fun.y=mean, geom="point", shape=23, size=2, color = 'skyblue') +
  labs(title = 'Violin plot of age grouped by sex') +
  theme_classic() +
  theme(text = element_text(size = 20))

ggplot(lung_data, aes(x=sex, y=age)) +
  geom_violin(trim=FALSE, fill="gray") +
  geom_boxplot(width=0.3) +
  labs(title = 'Violin plot of age grouped by sex') +
  theme_classic() +
  theme(text = element_text(size = 20))



In [ ]:
# scatter plot of weight loss by calories
ggplot(lung_data, aes(x=wt.loss, y=meal.cal)) +
  geom_point(size=2, shape=23) +
  labs(title = 'Scatter plot of weight loss by meal calories') +
  theme_classic() +
  theme(text = element_text(size = 20))

# Adding linear fit to the scatter plot and grouping by sex to visualise differences between men and women
ggplot(lung_data, aes(x=wt.loss, y=meal.cal, color=sex)) +
  geom_point(size=2, shape=19) +
  geom_smooth(method=lm, linetype="dashed", fill='skyblue') +
  labs(title = 'Scatter plot of weight loss by meal calories') +
  theme_classic() +
  theme(text = element_text(size = 20))


### 2b. Understanding the data
We can also explore summary statistics and identify significant differences between groups.

Consider the use of parametric tests (assume normality in the data) and non-parametric test (do not assume normality in the data). To test for normality you can use the **shapiro.test**, this compares the distribution to a normal distribution, a p>0.05 shows there is no significant difference from a normal distribution and parametric tests could be used.

For example - testing age in the cell below.


In [ ]:
shapiro.test(lung_data$age)

**Some common examples to get started, this list is not exhaustive!**

Examples for continuous variables:
*   students t-test for parametric data
*   Wilcoxon signed-rank test for non-parametric data

Testing against a catagorical variable could be done with an ANOVA

Correlations between variables can also be investiagated using:
*   Pearson
*   Spearman (non parameteric, ranks a monotonic relationship)





In [ ]:
# Some examples of the tests described above

t.test(age ~ sex, data = lung_data)
wilcox.test(age ~ sex, data = lung_data)

wilcox.test(wt.loss ~ sex, data = lung_data)


We can see there is no significant difference in age between male and females in the dataset.
We can, however, see a significant difference in weight loss. To get a fuller picture, we can use the next cell to provide a summary of the weight loss for males (sex = 1) and females (sex = 2), showing that males (in general) lose significantly more weight.


In [ ]:
tapply(lung_data$wt.loss, lung_data$sex, summary)

Testing catagorical data using an ANOVA test, this allows us to compare means in a situation where there are more than two groups. Here, using the multiple groups in the ECOG performance status.

In [ ]:
# Clean the data to remove the NA's in the performance status
# We will also remove the performance status 3 patient as there is only one with this value which will not result in any statistical power

lung_data <- lung_data %>%
  filter(ph.ecog %in% c(0,1,2))

summary(factor(lung_data$ph.ecog))

test_aov <- aov(wt.loss ~ ph.ecog, data = lung_data)
summary(test_aov)

So, in the above test, we can see that there is a significant in the groups, but we don't know between which groups (or indeed, possibly, between all groups). To check where we see significance we can use a Tukey test to comapre the means between all groups.   

In [ ]:
TukeyHSD(test_aov)

From the box above, we can see there is a significance difference in weight loss between patients with ECOG performance status 0 and 2.

Finally, we will look at how to stest for correlations between variables.

In [ ]:
cor.test(lung_data$meal.cal, lung_data$wt.loss, method = "pearson")
cor.test(lung_data$meal.cal, lung_data$wt.loss, method = "spearman")

So, no correlation between weight loss and meal calories in this example and the correlation coefficients are also very low. If you remember the scatter plot from above then this is not an unexpected result!




---
### 3. Survial analysis

Now, let's start building a survival function and plot our first survival graph. First, we will use the function **survfit** to create a survival function, we can also call this to provide summary statistics for the population.

(The survival function expects events to be 0 (no event) or 1 (event), the first lines of code changes the cataogories to work with the function.)

In [ ]:
#cleaning the data so teh survival event entries are as epected by the Surv function

lung_data_surv <- lung_data %>%
  mutate(status = case_when(status == 2 ~ 1,
                            status == 1 ~ 0))




First, let's look at the survival curve for the full cohort. In the survival function below, we denote this using the **~1**.

Printing the function (**s1**) will provide median survival times for the cohort, and if we specify groups, for each group in turn.


In [ ]:
s1 <- survfit(Surv(time, status) ~ 1, data = lung_data_surv)
s1


Now we have the **survfit** function, we can plot our first Kaplan Meier curve.

In [ ]:
ggsurvplot(s1, risk.table = TRUE, conf.int = TRUE, ncensor.plot = FALSE)



Now, let's look to see if men and women have different survival times.

In [ ]:
s2 <- survfit(Surv(time, status) ~ sex, data = lung_data_surv)
s2

In [ ]:
ggsurvplot(s2, risk.table = TRUE, conf.int = TRUE, surv.median.line = "hv", pval = TRUE,ncensor.plot = FALSE)


We can create different groups to test for survival differences, but be careful you don't start to p-hack....


In [ ]:
print(paste('Median calorie intake in population is', median(lung_data_surv$meal.cal, na.rm = TRUE), sep = ' '))

lung_data_surv$cal_split <- lung_data_surv$meal.cal > median(lung_data_surv$meal.cal, na.rm = TRUE)
#lung_data_surv$cal_split <- lung_data_surv$meal.cal > 398   ### this is me trying to p-hack!!!!

s3 <- survfit(Surv(time, status) ~ cal_split, data = lung_data_surv)
s3
ggsurvplot(s3, risk.table = TRUE, conf.int = TRUE, surv.median.line = "hv", pval = TRUE,ncensor.plot = FALSE)






---

### 4. Cox proportional hazards model
In this section we will start to build up a cox model to investigate univariable and multivariable models for survival.

First, let's build a uni- (one) variable cox model.


In [ ]:
univariable_cox <- coxph(Surv(time, status) ~ sex, data = lung_data_surv)

summary(univariable_cox)

If you remember from the lecture, the exp(coef) will give us the **hazard ratio** for the variable. The confidence intervals are also presented for the hazard ratio in the second row above.

In this case, age is significantly associated with survival. The hazard ratio is 0.6, which indicated the risk of death is decreasing as an individual becomes older.

The format for a multivariable model is the same, we just add any additional variables of interest after the **~** using a **+** between variables.

In [ ]:
multivariable_cox <- coxph(Surv(time, status) ~ sex + age + ph.ecog + wt.loss, data = lung_data_surv)

summary(multivariable_cox)

In the above table, look to see how continuous variables (age and weight loss) and catagorical variables (sex and ECOG performance status) are handled. For a catogorical variable, each catagory in turn is compared to the refernce catagory, default is the first. i.e., each ECOG is compared against ECOG = 0.


We can also present a visualisation of the hazard ratios, these figures are called forest plots. This will plot the hazard ratios, with the confidence interval shown. If the confidence intervals includes the value 1, this visually indicates this variable does not have a significance association with the outcome.


In [ ]:
ggforest(multivariable_cox)


---
### 4. Odds ratios

In this final section we will have a look at Odds ratios. As we discussed, the main difference between Odds ratios and the Cox proportional hazards models is that the Odds ratio does not handle time to an event.

Here, we will use the **status** variable as our event, **i.e., we don't care when the event happen, just that is has happend.** From our dataset, we will
set our exposure as being female and calculate the odd ratio for death of being female compared to being male.

In [ ]:
## Method 1 - very manual
## First we will extract the data into a matrix, showing us how many events are in each group

event_count_female <- lung_data_surv %>%
  filter(sex == 2) %>%
  group_by(status) %>%
  summarise(n = n())

event_count_male <- lung_data_surv %>%
  filter(sex == 1) %>%
  group_by(status) %>%
  summarise(n = n())

event_count <- event_count_female
event_count$tmp <- event_count_male[,2]

colnames(event_count) <- c('event', 'female', 'male')
event_count <- t(event_count)
event_count

In [ ]:
## Now let's calculate the odds of an event for each group
## want event yes / event no

odds_female = event_count[2,2] / event_count[2,1]
odds_male = event_count[3,2] / event_count[3,1]

odds_female
odds_male



In [ ]:
## And calculated the odds ratio
odds_ratio = odds_female / odds_male
odds_ratio

So, female participants in the cohort are 0.34 times less likely of risk of death in the study than males.

Let's try something simpler. We can use a linear model to calculate the odds ratio in two steps. Like above for the cox proportional hazards model, is we take the exponential of the coefficient(s) from the linear model we calculate the odds ratio.

Let's try...

In [ ]:
linear_model <- glm(status ~ sex, family = "binomial", data = lung_data_surv)
summary(linear_model)

exp(coefficients(linear_model))



So, for sex2 (i.e., female) the odds ratio is 0.34, same answer as above working manually!

The nice thing about the linear model is it can allow for multivariable models too.


---

